In [1]:
import os
os.getcwd()

'/media/puzan/NewVolume/MyFolder/research'

In [2]:
os.chdir("../")

In [13]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTranslateConfig:
    root_dir:Path
    tokenizer_file:Path
    src_lang:str
    tgt_lang:Path
    max_seq_len:int
    model_path:Path
    model_basename:Path
    epoch_name: int
    
    

In [4]:
from translator.constants import *
from translator.utils.common import read_yaml,create_directories

In [14]:
class ConfigurationManager:
    def __init__(self,config_file_path=CONFIG_FILE_PATH,params_file_path=PARAMS_FILE_PATH):
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)
        create_directories([self.config.artifacts_root])
    
    def get_model_translate_config(self)-> ModelTranslateConfig:
        config=self.config.data_translate
        params=self.params.modeltrainer
        print(len(params))
        create_directories([config.root_dir])
        model_translate_config=ModelTranslateConfig(
            root_dir=config.root_dir,
            tokenizer_file=config.tokenizer_file,
            src_lang=params.src_lang,
            tgt_lang=params.tgt_lang,
            max_seq_len=params.max_seq_len,
            model_path=config.model_path,
            model_basename=config.model_basename,
            epoch_name=config.epoch_name

        )
        return model_translate_config

In [11]:
import torch

In [6]:
from tokenizers import Tokenizer
from translator.model import build_transformer




In [17]:
class Load_model:
    def __init__(self,config:ModelTranslateConfig):
        self.config=config
        self.model=None
        self.tokenizer_src=None
        self.tokenizer_tgt=None
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    def get_weights_file_path(self,epoch:str):
        model_folder=self.config.model_path # extracting model folder from the config
        model_basename=self.config.model_basename# extracting the base name for model files
        model_filename=f'{model_basename}{epoch}.pt'
        return str(Path('.')/model_folder/model_filename)
    
    def load_model(self):
        
        self.tokenizer_src = Tokenizer.from_file(self.config.tokenizer_file.format(self.config.src_lang))
        self.tokenizer_tgt = Tokenizer.from_file(self.config.tokenizer_file.format(self.config.tgt_lang))
        vocab_src_len = self.tokenizer_src.get_vocab_size()
        vocab_tgt_len = self.tokenizer_tgt.get_vocab_size()
        self.model = build_transformer(vocab_src_len, vocab_tgt_len, self.config.max_seq_len, self.config.max_seq_len)
        self.model = self.model.to(self.device)
        
        model_filename = self.get_weights_file_path(f"epoch_{self.config.epoch_name}")
        state = torch.load(model_filename, map_location=self.device)
        self.model.load_state_dict(state['model_state_dict'])
        
    def preprocess_sentence(self,sentence, tokenizer, seq_len, sos_token, eos_token, pad_token):
        tokens = tokenizer.encode(sentence).ids
        num_padding_tokens = seq_len - len(tokens) - 2

        if num_padding_tokens < 0:
            raise ValueError("Sentence is too long")

        input_tensor = torch.cat([
            torch.tensor([sos_token], dtype=torch.int64),
            torch.tensor(tokens, dtype=torch.int64),
            torch.tensor([eos_token], dtype=torch.int64),
            torch.tensor([pad_token] * num_padding_tokens, dtype=torch.int64)
        ])

        return input_tensor.unsqueeze(0)

    def translate_sentence(self,sentence):
        sos_token = self.tokenizer_tgt.token_to_id('[SOS]')
        eos_token = self.tokenizer_tgt.token_to_id('[EOS]')
        pad_token = self.tokenizer_tgt.token_to_id('[PAD]')

        encoder_input = self.preprocess_sentence(sentence, self.tokenizer_src, self.config.max_seq_len, sos_token, eos_token, pad_token).to(self.device)
        encoder_mask = (encoder_input != pad_token).unsqueeze(1).unsqueeze(2).int().to(self.device)

        encoder_output = self.model.encode(encoder_input, encoder_mask)
        decoder_input = torch.tensor([[sos_token]], dtype=torch.int64).to(self.device)

        for _ in range(self.config.max_seq_len):
            decoder_mask = torch.triu(torch.ones((1, decoder_input.size(1), decoder_input.size(1))), diagonal=1).type_as(encoder_mask).to(self.device)
            decoder_output = self.model.decode(encoder_output, encoder_mask, decoder_input, decoder_mask)
            proj_output = self.model.project(decoder_output[:, -1])

            _, next_word = torch.max(proj_output, dim=1)
            decoder_input = torch.cat([decoder_input, next_word.unsqueeze(0)], dim=1)

            if next_word.item() == eos_token:
                break

        translated_tokens = decoder_input.squeeze(0).tolist()
        translated_text = self.tokenizer_tgt.decode(translated_tokens)
        return translated_text


        

        

In [20]:
try:
    config=ConfigurationManager()
    data_translate_config=config.get_model_translate_config()
    mod=Load_model(data_translate_config)
    mod.load_model()
    translated_sentence = mod.translate_sentence("This is a test sentence.")
    print(translated_sentence)
except Exception as e:
    raise e
    
    model_loader = Load_model(config)

# Load the model and tokenizers (only once

# Translate a sentence (no need to pass model or tokenizers again)


[2024-09-26 16:45:52,154:INFO:common:yaml_file:config/config.yaml loaded successfully
[2024-09-26 16:45:52,157:INFO:common:yaml_file:param/params.yaml loaded successfully
[2024-09-26 16:45:52,158:INFO:common:Directory 'artifacts' created successfully
7
[2024-09-26 16:45:52,159:INFO:common:Directory 'artifacts/data_translate' created successfully
torch.Size([1, 150, 512])
torch.Size([1, 150, 512])


/tmp/ipykernel_32504/162301265.py:24: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(model_filename, map_location=self.device)


पनि
